In [2]:
!pip install earthengine-api


   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   -- ------------------------------------- 1.0/14.6 MB 4.7 MB/s eta 0:00:03
   --------- ------------------------------ 3.4/14.6 MB 8.5 MB/s eta 0:00:02
   -------------------- ------------------- 7.3/14.6 MB 12.0 MB/s eta 0:00:01
   --------------------------------- ------ 12.3/14.6 MB 15.2 MB/s eta 0:00:01
   ---------------------------------------- 14.6/14.6 MB 15.7 MB/s  0:00:00

   --- ------------------------------------  1/13 [protobuf]
   --- ------------------------------------  1/13 [protobuf]
   --- ------------------------------------  1/13 [protobuf]
   --- ------------------------------------  1/13 [protobuf]
   --- ------------------------------------  1/13 [protobuf]
   --- ------------------------------------  1/13 [protobuf]
   --- ------------------------------------  1/13 [protobuf]
   --- ------------------------------------  1/13 [protobuf]
   --- ------------------------------------  1/13 [pro


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import ee
ee.Authenticate()
ee.Initialize()


Enter verification code:  4/1Ab32j932Z2WYm87PFdnG5wzWbonVNEOADhObAuUpWkOWTdHt_NBUEKWhZBY



Successfully saved authorization token.


EEException: Project huzie111 is not registered to use Earth Engine. Visit https://console.cloud.google.com/earth-engine/configuration?project=huzie111 to register your project. See https://developers.google.com/earth-engine/guides/access for more details. If you have previously registered your project for noncommercial use and lost access, you will need to re-verify your eligibility. See the https://developers.google.com/earth-engine/guides/access#configuring_noncommercial_access for more details.

In [6]:
# Initialize Earth Engine 
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()  # interactive; remove/replace with service-account flow if needed
    ee.Initialize()
    

In [8]:
# Parameters
import datetime

CHIRPS_ID = "UCSB-CHG/CHIRPS/DAILY"
chirps = ee.ImageCollection(CHIRPS_ID).select("precipitation")

# CHIRPS earliest date (dataset starts in 1981). The user asked for 1970,
# but CHIRPS doesn't cover that; adjust automatically.
requested_start = datetime.date(1970, 1, 1)
dataset_start = datetime.date(1981, 1, 1)  # CHIRPS availability
start_date = max(requested_start, dataset_start)
end_date = datetime.date.today()

if requested_start < dataset_start:
    print(
        f"Requested start {requested_start} is earlier than CHIRPS availability; "
        f"using {dataset_start} instead."
    )

# Get a country geometry for Uganda using the diplomatic boundaries dataset.
# USDOS/LSIB_SIMPLE/2017 has a 'country_na' property with country name.
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
uganda_fc = countries.filter(ee.Filter.eq("country_na", "Uganda"))

# Sanity check: ensure we have a feature
count = uganda_fc.size().getInfo()
if count == 0:
    raise ValueError("Uganda feature not found in USDOS/LSIB_SIMPLE/2017.")
uganda = uganda_fc.first()  # ee.Feature
uganda_geom = uganda.geometry()

# Resolution (m). CHIRPS native resolution ~0.05° (~5.5 km). Use 5000 m as scale.
SCALE = 5000

# Helper: iterate months from start_date to end_date (inclusive of month containing end_date)
def month_ranges(start_date, end_date):
    """Yield (month_start_date, month_end_date) pairs for each calendar month."""
    cur = datetime.date(start_date.year, start_date.month, 1)
    while cur <= end_date:
        # next month start
        if cur.month == 12:
            nxt = datetime.date(cur.year + 1, 1, 1)
        else:
            nxt = datetime.date(cur.year, cur.month + 1, 1)
        yield cur, nxt
        cur = nxt


Requested start 1970-01-01 is earlier than CHIRPS availability; using 1981-01-01 instead.


In [10]:
# Build monthly time series 
import time

records = []
month_iter = list(month_ranges(start_date, end_date))
print(f"Computing monthly totals for {len(month_iter)} months from {start_date} to {end_date} ...")

for month_start, month_end in month_iter:
    # Filter daily images in the month and sum them -> monthly total (mm)
    img_month = chirps.filterDate(str(month_start), str(month_end)).sum()

    # Reduce over the Uganda geometry to get mean monthly precipitation (mm)
    # We use ee.Reducer.mean() which gives the mean pixel value (monthly total) across all pixels.
    # That is an approximation of the country-average; for an exact area-weighted mean you can
    # multiply by pixel area and divide by total area. For many use-cases mean is acceptable.
    try:
        stat = img_month.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=uganda_geom,
            scale=SCALE,
            crs="EPSG:4326",
            maxPixels=1e13,
        )
        mean_monthly = stat.get("precipitation").getInfo()
    except Exception as e:
        # Catch transient EE server issues and proceed with None
        print(f"Warning: reduceRegion failed for {month_start}: {e}")
        mean_monthly = None

    records.append({"date": month_start.isoformat(), "mean_monthly_precip_mm": mean_monthly})

    # small delay to be polite to EE servers for repeated getInfo calls
    time.sleep(0.2)


Computing monthly totals for 539 months from 1981-01-01 to 2025-11-26 ...


In [13]:
# Convert to pandas DataFrame and save CSV
import pandas as pd
df = pd.DataFrame(records)
csv_out = "uganda_chirps_monthly_mean_precip.csv"
df.to_csv(csv_out, index=False)
print(f"Saved monthly country-average precipitation to: {csv_out}")

# Example: print first few rows
print(df.head(10))

# If you want annual totals instead of monthly, aggregate monthly results:
df["date"] = pd.to_datetime(df["date"])
df["year"] = df["date"].dt.year
annual = df.groupby("year", dropna=True)["mean_monthly_precip_mm"].sum().reset_index()
annual_csv = "uganda_chirps_annual_total_precip.csv"
annual.to_csv(annual_csv, index=False)
print(f"Saved annual country-total precipitation (approx) to: {annual_csv}")
print(annual.head(10))

Saved monthly country-average precipitation to: uganda_chirps_monthly_mean_precip.csv
         date  mean_monthly_precip_mm
0  1981-01-01               22.015856
1  1981-02-01               33.893070
2  1981-03-01              160.400762
3  1981-04-01              167.870541
4  1981-05-01              132.536484
5  1981-06-01               66.933945
6  1981-07-01              112.616224
7  1981-08-01              144.071411
8  1981-09-01              135.637042
9  1981-10-01              126.913703
Saved annual country-total precipitation (approx) to: uganda_chirps_annual_total_precip.csv
   year  mean_monthly_precip_mm
0  1981             1227.236356
1  1982             1265.669795
2  1983             1112.222286
3  1984             1062.280319
4  1985             1214.091055
5  1986             1126.936859
6  1987             1182.241946
7  1988             1309.887029
8  1989             1250.649806
9  1990             1233.385988
